# Notebook Etape 3 - Système de recommendations / MachineLearning.

In [37]:
import pandas as pd
import unicodedata
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler

#### Import CSV de la Sélection de films :

In [38]:
DF = pd.read_csv(r'C:\Users\33631\Documents\Export_CSV\Notebook_Etape_1_Terminado.csv', sep=',')

### 1. Préparation de la DataFrame au Machine Learning.

In [39]:
DF["genres"] = DF["genres"].str.split(",")                      # Dissociation des genres en catégories uniques
Ex = DF.explode('genres')
GenreF = pd.concat([Ex['genres'].str.get_dummies()],axis=1)     # Création d'une nouvelle DF avec chaque genre détaillé dans sa colonne respective
GenreF = GenreF.groupby(GenreF.index).sum() 
DFM = pd.concat([DF, GenreF],axis=1)                            # Ajout des colonnes à notre DF principal
DFM1 = DFM.drop(columns=['actor', 'director'])                  # Tri des colonnes superflues
DFM1.dropna(subset=['startYear'], inplace=True)                 # Suppression des valeurs NaN 
DFM1.dropna(subset=['genres'], inplace=True)                    # Suppression des valeurs NaN 
DFM1['startYear'] = DFM1['startYear'].astype(int)               # Changement de type de la colonne 'startYear' en 'int'
DFM1['titrePropre'] = DFM1['originalTitle'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.lower().str.replace(' ', '')
                                                                # Création d'une colonne de primaryTitle sans accents, majuscules ou espaces
DFM1.reset_index(drop= True, inplace = True)                    # Réinitialisation de l'index après la supression de quelques lignes de notre DF

### 2. Standardisation des données + Application au modèle.

In [40]:
X = DFM1.select_dtypes("number")                                                # Choix des valeurs numériques uniquement dans notre DF
scaler = StandardScaler()                                                       # Appel de la fonction StandardScaler()
scaler.fit(X)                                                                   # Application de la fonction à notre DF
X_scaled = pd.DataFrame(scaler.transform(X), index=X.index, columns=X.columns)
modelNN = NearestNeighbors(n_neighbors=6).fit(X_scaled)                         # Application de notre DF (après standardisation) à notre modèle de ML 'NearestNeighbors'

### 3. Automatisation du modèle.

In [41]:
def Recherche():                                                                                                                # Première fonction d'automatisation du modèle
    pninput = input('Recherche')                                                                                                # Premier user input de recherche de film
    pninput = unicodedata.normalize('NFKD', pninput).encode('ascii', errors='ignore').decode('utf-8').lower().replace(' ', '')  # Uniformisation de son input sans accents, majuscules, espaces
    selectionInput = DFM1[DFM1['titrePropre'].str.contains(pninput)]
    selectionInput2 = selectionInput[['originalTitle', 'startYear']]                                                            # Choix des colonnes sur lesquelles travailler de notre DF
    if len(selectionInput2) == 1 :                                                                                              # Premier choix de notre modèle, s'il n'y a qu'une seule référence correspondante => return R1
        R1 = selectionInput2.iloc[0]
        return R1                                                                                                               # Si cas de figure 1, le modèle passe sur la deuxième fonction.
    elif len(selectionInput2) >= 2 :                                                                                            # Deuxième choix de notre modèle, s'il y a plusieurs références correspondantes, nouvel user input
        print(selectionInput2.set_index(pd.Index(range(1, len(selectionInput2)+1))).to_string(header=False))
        SecInput = int(input("Veuillez saisir l'ID du film correspondant à votre recherche :"))                                 # Deuxième user input afin de préciser le bon film recherché par son numéro d'index affiché
        R1 = selectionInput2.iloc[SecInput-1]
        return R1                                                                                                               # Si cas de figure 2, le modèle passe sur la deuxième fonction.
    else :
        print("Navré, nous n'avons trouvé aucune référence correspondante")                                                     # Sinon le modèle s'arrête là.
        return None

# ---------------------------------------------------------------------------------------------------------------------------- #

def Recommandation():                                                                                                           # Deuxième fonction d'automatisation du modèle
    R1 = Recherche()                                                                                                            # Rappel de la première fonction et de son résultat
    if R1 is not None :                                                                                                         # Si un résultat existe :
        indiv_concerne = X_scaled.loc[R1.name].to_frame().T                                                                     # Le modèle isole le titre du film choisi par l'utilisateur et prends ses valeurs comme référence 
        neigh_dist, neigh_cli = modelNN.kneighbors(indiv_concerne, n_neighbors=6)                                               # Le modèle trouve les 6 titres les plus proches (en l'incluant) en comparant les valeurs
        cli_ressem = neigh_cli[0][1:]                                                                                           # Il les stocke dans une variable en excluant le titre de référence
        Result = DFM1.iloc[cli_ressem]                                                                                          # Il peut nous ressortir les noms des films qui nous intéressent dans la DF initiale 
        index = pd.Index((range(1, len(Result)+1)))
        Result = Result.set_index(index)                                                                                        # Modification de l'affichage du résultat        
        print()
        print("Vous avez selectionné :", R1['originalTitle'], '(', R1['startYear'], ')')
        print("Nous vous recommandons :", Result[['originalTitle', 'startYear']].to_string(header=False), sep='\n')             # Affichage du résultat 
    else :
        pass


### 4. Résultats :

In [61]:
Recommandation()                                                                                                                    

1  Harry Potter and the Deathly Hallows: Part 2  2011
2      Harry Potter and the Prisoner of Azkaban  2004
3  Harry Potter and the Deathly Hallows: Part 1  2010
4           Harry Potter and the Goblet of Fire  2005
5         Harry Potter and the Sorcerer's Stone  2001
6        Harry Potter and the Half-Blood Prince  2009
7     Harry Potter and the Order of the Phoenix  2007
8       Harry Potter and the Chamber of Secrets  2002

Vous avez selectionné : Harry Potter and the Prisoner of Azkaban ( 2004 )
Nous vous recommandons :
1           Harry Potter and the Goblet of Fire  2005
2       Harry Potter and the Chamber of Secrets  2002
3  Harry Potter and the Deathly Hallows: Part 1  2010
4         Harry Potter and the Sorcerer's Stone  2001
5       Fantastic Beasts and Where to Find Them  2016
